In [10]:
#1先导包
import torch
from torch import nn
from model import TextRNN
from cnews_loader import read_vocab,read_category,process_file 
from torch import optim
import numpy as np

In [11]:
#2导入数据
train_file='cnews.train.txt'
print("train_file：\n{}".format(train_file))

test_file='cnews.test.txt'
print("test_file：\n{}".format(test_file))

val_file='cnews.val.txt'
print("val_file：\n{}".format(val_file))

vocab_file='cnews.vocab.txt'
print("vocab_file：\n{}".format(vocab_file))

train_file：
cnews.train.txt
test_file：
cnews.test.txt
val_file：
cnews.val.txt
vocab_file：
cnews.vocab.txt


In [24]:
def train():
    model=TextRNN()
    #定义损失函数
    Loss=nn.MultiLabelSoftMarginLoss()
    optimizer=optim.Adam(model.parameters(),lr=0.001)
    
    #先定义一个最好的分数
    best_val_acc=0
    #开始训练
    for epoch in range(1000):
        print('第{}次迭代：'.format(epoch))
        #分批训练
        for step,(x_batch,y_batch) in enumerate(train_loader):
            x=x_batch
            y=y_batch
            out=model(x)
            loss=Loss(out,y)
            print("当前的loss是：{}".format(loss))
            
            #反向传播
            optimizer.zero_grad()
            optimizer.step()
            accuracy=np.mean( (torch.argmax(out,1)==torch.argmax(y,1) ).cpu().numpy())
            print("准确率：",accuracy)
        #对模型进行验证
        if (epoch+1)%5==0:
            for step,(x_batch,y_batch) in enumerate(val_loader):
                x=x_batch.cuda()
                y=y_batch.cuda()
                out=model(x)
                accuracy=np.mean(torch.argmax(out,1)==torch.argmax(y,1)).cpu().numpy()
                if accuracy>best_val_acc:
                    torch.save(model,"songhaoMode.pkl")#保存模型
                    best_val_acc=accuracy
                    print("当前模型已经保存")
                    print("此时的准确率是:{}".format(accuracy))                

In [13]:
# 获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
print(categories)

# 获取训练文本中所有出现过的字及其所对应的id
words, word_to_id = read_vocab('cnews.vocab.txt')
print(words)

# 获取训练数据每个字的id和对应标签的one-hot形式
x_train, y_train = process_file(train_file, word_to_id, cat_to_id, 600)
print('x_train=', x_train)
x_val, y_val = process_file(val_file, word_to_id, cat_to_id, 600)

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
['<PAD>', '，', '的', '。', '一', '是', '在', '0', '有', '不', '了', '中', '1', '人', '大', '、', '国', '', '2', '这', '上', '为', '个', '“', '”', '年', '学', '时', '我', '地', '和', '以', '到', '出', '来', '会', '行', '发', '：', '对', '们', '要', '生', '家', '他', '能', '也', '业', '金', '3', '成', '可', '分', '多', '现', '5', '就', '场', '新', '后', '于', '下', '日', '经', '市', '前', '过', '方', '得', '作', '月', '最', '开', '房', '》', '《', '高', '9', '8', '.', '而', '比', '公', '4', '说', ')', '将', '(', '都', '资', 'e', '6', '基', '用', '面', '产', '还', '自', '者', '本', '之', '美', '很', '同', '', '7', '部', '进', '但', '主', '外', '动', '机', '元', '理', '加', 'a', '全', '与', '实', '影', '好', '小', '间', '其', '天', '定', '表', '力', '如', '次', '合', '长', 'o', '体', '价', 'i', '所', '内', '子', '目', '电', '-', '当', '度', '品', '看', '期', '关', '更', 'n', '等', '工', '然', '斯', '重', '些', '球', '此', '里', '利', '相', '情', '投', '点', '没', '因', '已', '三', '心', '特', '明', '位', '两', '法', '从', '入', '名', '万', '手', '计', '性', '事', '只', '样', '示', 'r', '

In [17]:
#设置一下GPU
cuda=torch.device('cuda')
x_train,y_train=torch.LongTensor(x_train),torch.Tensor(y_train)
x_val,y_val=torch.LongTensor(x_val),torch.Tensor(y_val)

import torch.utils.data as Data
train_dataset=Data.TensorDataset(x_train,y_train)
train_loader=Data.DataLoader(dataset=train_dataset,batch_size=128,shuffle=True)
val_dataset=Data.TensorDataset(x_val,y_val)
val_loader=Data.DataLoader(dataset=val_dataset,batch_size=128 )

In [25]:
train()

第0次迭代：
当前的loss是：0.7347633838653564
准确率： 0.0703125
当前的loss是：0.73409104347229
准确率： 0.109375
当前的loss是：0.734524667263031
准确率： 0.09375
当前的loss是：0.7344620227813721
准确率： 0.15625
当前的loss是：0.7342677116394043
准确率： 0.1015625
当前的loss是：0.7346452474594116
准确率： 0.1171875
当前的loss是：0.7348431944847107
准确率： 0.046875
当前的loss是：0.7342806458473206
准确率： 0.1171875
当前的loss是：0.734515905380249
准确率： 0.109375
当前的loss是：0.7347528338432312
准确率： 0.078125
当前的loss是：0.7347445487976074
准确率： 0.078125
当前的loss是：0.7345641851425171
准确率： 0.078125
当前的loss是：0.7347750663757324
准确率： 0.0859375
当前的loss是：0.7345267534255981
准确率： 0.0546875
当前的loss是：0.7346035242080688
准确率： 0.09375
当前的loss是：0.7346878051757812
准确率： 0.109375
当前的loss是：0.7342379093170166
准确率： 0.1171875
当前的loss是：0.7345521450042725
准确率： 0.15625
当前的loss是：0.7346546649932861
准确率： 0.09375
当前的loss是：0.7348941564559937
准确率： 0.078125
当前的loss是：0.7344029545783997
准确率： 0.078125
当前的loss是：0.7342884540557861
准确率： 0.1328125
当前的loss是：0.7344810366630554
准确率： 0.1015625
当前的loss是：0.734557032585144


AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx